In [1]:
import numpy as np
import pandas as pd

import operator
import random
import math

from dataclasses import dataclass
from functools import partial

from deap import gp, base, creator, tools, algorithms

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score



In [2]:
PATH_TRAIN = "./data/train.csv"
PATH_TEST = "./data/test.csv"

DTYPES_FEATURES = {
          "id": "uint64",
          "fr_COO": "category",
          "fr_COO2": "category",
      }

DTYPES_TARGETS = {
          "EC1": "bool",
          "EC2": "bool",
          "EC3": "bool",
          "EC4": "bool",
          "EC5": "bool",
          "EC6": "bool"
}

DROP_COLS = ["EC3", "EC4", "EC5", "EC6"]



def _load_data(datapath: str, dtypes: dict, drop_cols: list) -> pd.DataFrame:
  return pd.read_csv(
      filepath_or_buffer=datapath,
      dtype=dtypes,
      index_col="id"
    ).drop(columns=drop_cols, axis=1)


GetTrainDF = partial(_load_data, datapath=PATH_TRAIN, dtypes=dict(**DTYPES_TARGETS, **DTYPES_FEATURES), drop_cols=DROP_COLS)
GetTestDF = partial(_load_data, datapath=PATH_TEST, dtypes=DTYPES_FEATURES, drop_cols=[])

df_train = GetTrainDF().astype(float)
df_train

,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,EState_VSA2,...,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2
id,,,,,,,,,,,,,,,,,,,,,
0,323.390782,9.879918,5.875576,5.875576,4.304757,4.304757,2.754513,1.749203,0.000000,11.938294,...,0.000000,0.000000,17.744066,0.000000,4.794537,35.527357,0.0,0.0,1.0,1.0
1,273.723798,7.259037,4.441467,5.834958,3.285046,4.485235,2.201375,1.289775,45.135471,0.000000,...,0.000000,0.000000,7.822697,30.705892,13.825658,44.707310,0.0,0.0,0.0,1.0
2,521.643822,10.911303,8.527859,11.050864,6.665291,9.519706,5.824822,1.770579,15.645394,6.606882,...,53.378235,0.000000,15.645394,73.143616,17.964475,45.660120,0.0,0.0,1.0,1.0
3,567.431166,12.453343,7.089119,12.833709,6.478023,10.978151,7.914542,3.067181,95.639554,0.000000,...,0.000000,6.420822,15.645394,62.107304,31.961948,87.509997,0.0,0.0,1.0,1.0
4,112.770735,4.414719,2.866236,2.866236,1.875634,1.875634,1.036450,0.727664,17.980451,12.841643,...,19.386400,0.000000,11.938611,18.883484,9.589074,33.333333,2.0,2.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14833,632.207041,10.911303,6.579933,9.179964,4.653583,6.030052,3.670528,1.770579,32.971529,6.606882,...,13.847474,6.923737,34.407699,32.607024,18.947452,61.376610,0.0,0.0,1.0,1.0
14834,62.568425,2.642734,1.446898,1.446898,0.879497,0.879497,0.174620,0.000000,0.000000,0.000000,...,0.000000,6.066367,0.000000,6.420822,0.000000,10.000000,0.0,0.0,0.0,1.0
14835,981.327476,10.363081,6.146219,6.146219,4.700576,4.700576,3.064846,2.133897,17.248535,0.000000,...,0.000000,23.762553,10.969244,0.000000,0.000000,66.666667,0.0,0.0,1.0,1.0


In [3]:
# names (=argn) and number (=argv) of arguments

argn = df_train.drop(columns=["EC1", "EC2"], inplace=False).columns.to_list()
argc = len(argn)

In [4]:
pset = gp.PrimitiveSet("MAIN", arity=argc, prefix="ARG")

pset.renameArguments(**{f"ARG{i}": arg for i, arg in enumerate(argn)})

def protectedDiv(left, right):
    if right == 0:
        return 1
    else:
         return left / right

pset.addPrimitive(operator.add, 2, name="add")
pset.addPrimitive(operator.sub, 2, name="sub")
pset.addPrimitive(operator.mul, 2, name="mul")
pset.addPrimitive(protectedDiv, 2, name="div")
pset.addPrimitive(operator.neg, 1, name="neg")
pset.addPrimitive(math.sin, 1, name="sin")
pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))

In [5]:
# create a fitness and individual

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

In [6]:
# create a toolbox

toolbox = base.Toolbox()


In [7]:
# create a X_train, y_train, X_test, y_test

X = df_train.drop(columns=["EC1", "EC2"], inplace=False).values
y = df_train["EC1"].values.astype(int)


In [8]:
# create a fitness function that takes an individual as input and returns the corresponding auc score

def evalBinaryClassification(individual, X, y):
    
    func = gp.compile(expr=individual, pset=pset)
    y_pred_vals = np.array([func(*x) for x in X])
    # replace all values < 0 with 0, and all values > 1 with 1, nan with 0
    y_pred_vals = np.nan_to_num(y_pred_vals, copy=True, nan=0.0, posinf=1.0, neginf=0.0)
    y_pred_vals[y_pred_vals < 0] = 0
    y_pred_vals[y_pred_vals > 1] = 1

    return roc_auc_score(y, y_pred_vals.astype(int)),


In [9]:
# set up the toolbox for the gp algorithm

toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=4)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalBinaryClassification, X=X, y=y)

# bloat control
toolbox.decorate(
    "mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17)
)
toolbox.decorate(
    "mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17)
)

In [10]:
# create and run the algorithm for target EC1

from deap import algorithms

NGEN = 30
POPSIZE = 500
CXPB = 0.9
MUTPB = 0.05

pop = toolbox.population(n=POPSIZE)
hof_ec1 = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)


pop, log = algorithms.eaSimple(pop, toolbox, CXPB, MUTPB, NGEN, stats=stats, halloffame=hof_ec1, verbose=True)

# print the best individual
print(hof_ec1[0])


# save the winner program
f_ec1_raw = gp.compile(hof_ec1[0], pset)
def f_ec1(args):
    return round(f_ec1_raw(*args))

gen	nevals	avg     	std      	min     	max     
0  	500   	0.497592	0.0285181	0.397719	0.586453
1  	458   	0.505519	0.0300165	0.397719	0.594594
2  	456   	0.512291	0.0348349	0.397719	0.594594
3  	445   	0.525955	0.0374825	0.402441	0.599093
4  	463   	0.539106	0.040628 	0.39111 	0.601631
5  	448   	0.554813	0.0383742	0.405136	0.605753
6  	434   	0.561066	0.0408734	0.421313	0.605753
7  	447   	0.561819	0.0411907	0.415644	0.620574
8  	434   	0.561922	0.0440328	0.40828 	0.617386
9  	472   	0.563123	0.0446969	0.408909	0.61871 
10 	433   	0.565614	0.0431136	0.402009	0.61372 
11 	444   	0.569433	0.0411814	0.411388	0.618459
12 	466   	0.566763	0.0464646	0.402013	0.613513
13 	451   	0.569764	0.0437855	0.421313	0.617552
